# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [39]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [40]:
csvfile = os.path.join('..', 'WeatherPy', 'citiesdf.csv')

citiesdf = pd.read_csv(csvfile)
citiesdf.head()

,Unnamed: 0,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,mataura,-46.1927,168.8643,285.93,83.0,93.0,0.89,NZ,1.616172e+09
1,1,ushuaia,-54.8000,-68.3000,284.15,62.0,40.0,5.66,AR,1.616172e+09
2,2,wanning,18.8003,110.3967,296.26,95.0,15.0,2.66,CN,1.616172e+09
3,3,palabuhanratu,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,atuona,-9.8000,-139.0333,299.87,77.0,48.0,6.03,PF,1.616172e+09


In [60]:
#drop NaNs
dropdupcities = citiesdf.dropna(subset=['Latitude', 'Longitude'])

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [61]:
gmaps.configure(api_key=g_key)

In [62]:
humidity = dropdupcities['Humidity']
locate = dropdupcities[['Latitude', 'Longitude']]

fig = gmaps.figure()
heatmap = gmaps.heatmap_layer(locate, weights=humidity, dissipating=False)
heatmap.max_intensity= 100
heatmap.point_radius = 5

fig.add_layer(heatmap)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [66]:
new_weatherdf = citiesdf[citiesdf['Max Temperature'] > 70]
new_weatherdf = citiesdf[citiesdf['Max Temperature'] <= 80]
new_weatherdf = citiesdf[citiesdf['Wind Speed'] <= 10]
new_weatherdf = citiesdf[citiesdf['Cloudiness'] == 0]
new_weatherdf_filter = new_weatherdf.dropna()
new_weatherdf_filter.head()

,Unnamed: 0,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
8,8,altay,47.8667,88.1167,262.32,45.0,0.0,2.35,CN,1.616172e+09
11,11,nizhnevartovsk,60.9344,76.5531,272.15,55.0,0.0,4.00,RU,1.616172e+09
14,14,najran,17.4924,44.1277,302.15,13.0,0.0,4.12,SA,1.616172e+09
19,19,mar del plata,-38.0023,-57.5575,293.15,49.0,0.0,6.17,AR,1.616172e+09
23,23,faya,18.3851,42.4509,292.65,77.0,0.0,2.57,SA,1.616172e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [69]:
hotel_df = pd.DataFrame(new_weatherdf_filter)
hotel_df['Hotel Name'] = ""
hotel_df.head()

,Unnamed: 0,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
8,8,altay,47.8667,88.1167,262.32,45.0,0.0,2.35,CN,1.616172e+09,
11,11,nizhnevartovsk,60.9344,76.5531,272.15,55.0,0.0,4.00,RU,1.616172e+09,
14,14,najran,17.4924,44.1277,302.15,13.0,0.0,4.12,SA,1.616172e+09,
19,19,mar del plata,-38.0023,-57.5575,293.15,49.0,0.0,6.17,AR,1.616172e+09,
23,23,faya,18.3851,42.4509,292.65,77.0,0.0,2.57,SA,1.616172e+09,


In [70]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

In [79]:
for index, row in hotel_df.iterrows():
    lat = row['Latitude']
    lng = row['Longitude']
    
    params = {
        "keyword": "hotel",
        "radius": 5000,
        "key": g_key,
        }
    
    params['location'] = f"{lat}, {lng}"
    hotel_json = requests.get(base_url, params=params).json()
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_json["results"][0]["name"]
    except (KeyError, IndexError):
        print('skip')
hotel_df

skip
skip
skip
skip
skip
skip
skip
skip
skip
skip


,Unnamed: 0,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
8,8,altay,47.8667,88.1167,262.32,45.0,0.0,2.35,CN,1.616172e+09,阿勒泰市金都酒店
11,11,nizhnevartovsk,60.9344,76.5531,272.15,55.0,0.0,4.00,RU,1.616172e+09,Park-Hotel
14,14,najran,17.4924,44.1277,302.15,13.0,0.0,4.12,SA,1.616172e+09,
19,19,mar del plata,-38.0023,-57.5575,293.15,49.0,0.0,6.17,AR,1.616172e+09,NH Gran Hotel Provincial
23,23,faya,18.3851,42.4509,292.65,77.0,0.0,2.57,SA,1.616172e+09,
31,31,iquique,-20.2208,-70.1431,294.15,64.0,0.0,4.12,CL,1.616172e+09,Holiday Inn Express Iquique
36,36,ayr,55.4627,-4.6339,284.15,71.0,0.0,4.63,GB,1.616172e+09,Brig o' Doon House Hotel
41,41,necochea,-38.5473,-58.7368,294.82,50.0,0.0,2.24,AR,1.616172e+09,Hostel La casa de al lado
47,47,dikson,73.5069,80.5464,237.71,92.0,0.0,2.09,RU,1.616172e+09,Sberbank Rossii
50,50,new norfolk,-42.7826,147.0587,284.63,71.0,0.0,0.76,AU,1.616172e+09,The Woodbridge


In [81]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [84]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))